In [1]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)
import seaborn as sns
sns.set_theme(style="darkgrid")
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from scipy.stats import uniform

https://www.kaggle.com/competitions/icr-identify-age-related-conditions/data

In [2]:
train = pd.read_csv('train.csv', index_col=0)
train.shape

(617, 57)

In [3]:
X = train.drop(columns = ["Class"])
y = train["Class"]

Vamos seguir o exemplo de código encontrado em https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html.

In [11]:
numeric_features = X.select_dtypes(include=['float']).columns.to_list()
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

categorical_features = X.select_dtypes(include=['object']).columns.to_list()
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(drop='first')),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [12]:
clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("forest", RandomForestClassifier())
    ]
)

In [13]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['AB', 'AF', 'AH', 'AM', 'AR',
                                                   'AX', 'AY', 'AZ', 'BC',
                                                   'BD ', 'BN', 'BP', 'BQ',
                                                   'BR', 'BZ', 'CB', 'CC',
                                                   'CD ', 'CF', 'CH', 'CL',
                                                   'CR', 'CS', 'CU', 'CW ',
                                                   'DA', 'DE', 'DF', 'DH', 'DI', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['EJ'])])),
                ('forest', RandomForestClassifier())])

In [14]:
param_grid = {
    'forest__n_estimators': [1000],
    'forest__criterion': ['log_loss', 'gini'],
    'forest__max_depth': [None],
    'forest__min_samples_split': [2],
    'forest__min_samples_leaf': [1],
    'forest__min_weight_fraction_leaf': [0.0],
    'forest__max_features': uniform(),
    'forest__max_leaf_nodes': [None],
    'forest__min_impurity_decrease': [0.0],
    'forest__bootstrap': [True],
    'forest__oob_score': [False],
    'forest__n_jobs': [-1],
    'forest__random_state': [42],
    'forest__verbose': [10],
    'forest__warm_start': [True],
    'forest__class_weight': [{0: 1, 1: 1}, {0: 1, 1: 2}],
    'forest__ccp_alpha': [0.0],
    'forest__max_samples': [None],
}

In [15]:
metrics = ['accuracy', 'f1', 'neg_log_loss', 'precision', 'recall', 'roc_auc',]

In [16]:
grid = RandomizedSearchCV(clf,
                    param_distributions=param_grid,
                    n_iter = 50,
                    scoring=metrics,
                    random_state = 42,
                    n_jobs = -1,
                    verbose=10,
                    refit="neg_log_loss",
                    return_train_score=True)

In [17]:
grid.fit(X, y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    0.1s


building tree 1 of 1000building tree 2 of 1000

building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 281 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 306 tasks      | elapsed:    0.3s


building tree 166 of 1000building tree 167 of 1000

building tree 168 of 1000
building tree 169 of 1000
building tree 170 of 1000
building tree 171 of 1000
building tree 172 of 1000
building tree 173 of 1000
building tree 174 of 1000
building tree 175 of 1000
building tree 176 of 1000
building tree 177 of 1000
building tree 178 of 1000
building tree 179 of 1000
building tree 180 of 1000
building tree 181 of 1000
building tree 182 of 1000
building tree 183 of 1000
building tree 184 of 1000
building tree 185 of 1000
building tree 186 of 1000
building tree 187 of 1000
building tree 188 of 1000
building tree 189 of 1000
building tree 190 of 1000
building tree 191 of 1000
building tree 192 of 1000
building tree 193 of 1000
building tree 194 of 1000
building tree 195 of 1000
building tree 196 of 1000
building tree 197 of 1000
building tree 198 of 1000
building tree 199 of 1000
building tree 200 of 1000
building tree 201 of 1000
building tree 202 of 1000
building tree 203 of 1000
building tre

...

building tree 682 of 1000
building tree 683 of 1000
building tree 684 of 1000
building tree 685 of 1000
building tree 686 of 1000
building tree 687 of 1000
building tree 688 of 1000
building tree 689 of 1000
building tree 690 of 1000
building tree 691 of 1000
building tree 692 of 1000
building tree 693 of 1000
building tree 694 of 1000
building tree 695 of 1000
building tree 696 of 1000
building tree 697 of 1000
building tree 698 of 1000
building tree 699 of 1000
building tree 700 of 1000
building tree 701 of 1000
building tree 702 of 1000
building tree 703 of 1000
building tree 704 of 1000
building tree 705 of 1000
building tree 706 of 1000
building tree 707 of 1000
building tree 708 of 1000
building tree 709 of 1000
building tree 710 of 1000
building tree 711 of 1000
building tree 712 of 1000
building tree 713 of 1000
building tree 714 of 1000
building tree 715 of 1000
building tree 716 of 1000
building tree 717 of 1000
building tree 718 of 1000
building tree 719 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 729 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 809 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 850 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 893 tasks      | elapsed:    1.0s


building tree 852 of 1000
building tree 853 of 1000
building tree 854 of 1000
building tree 855 of 1000
building tree 856 of 1000
building tree 857 of 1000
building tree 858 of 1000
building tree 859 of 1000
building tree 860 of 1000
building tree 861 of 1000
building tree 862 of 1000
building tree 863 of 1000
building tree 864 of 1000
building tree 865 of 1000
building tree 866 of 1000
building tree 867 of 1000
building tree 868 of 1000
building tree 869 of 1000
building tree 870 of 1000
building tree 871 of 1000
building tree 872 of 1000
building tree 873 of 1000
building tree 874 of 1000
building tree 875 of 1000
building tree 876 of 1000
building tree 877 of 1000
building tree 878 of 1000
building tree 879 of 1000
building tree 880 of 1000
building tree 881 of 1000
building tree 882 of 1000
building tree 883 of 1000
building tree 884 of 1000
building tree 885 of 1000
building tree 886 of 1000
building tree 887 of 1000
building tree 888 of 1000
building tree 889 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 936 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    1.1s finished


RandomizedSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['AB',
                                                                                'AF',
                                                                                'AH',
                                                                                'AM',
                                                                                'AR',
                                                                                'AX',
                                                                                'AY',
                                                                                'AZ',
                                                                                'BC',
                                                                                'BD ',
                                                                                'BN',
                                                                                'BP',
                                                                                'BQ',
                                                                                'BR',
                                                                                'BZ',
                                                                                'CB',
                                                                                'CC',
                                                                                'CD ',
                                                                                'CF',
                                                                                'CH',
                                                                                'CL',
                                                                                'CR',
                                                                                'CS',
                                                                                'CU',
                                                                                'CW ',
                                                                                'DA',
                                                                                'DE',
                                                                                'DF',
                                                                                'DH',
                                                                                'DI', ...]),
                                                                              ('...
                                        'forest__min_samples_split': [2],
                                        'forest__min_weight_fraction_leaf': [0.0],
                                        'forest__n_estimators': [1000],
                                        'forest__n_jobs': [-1],
                                        'forest__oob_score': [False],
                                        'forest__random_state': [42],
                                        'forest__verbose': [10],
                                        'forest__warm_start': [True]},
                   random_state=42, refit='neg_log_loss',
                   return_train_score=True,
                   scoring=['accuracy', 'f1', 

In [18]:
pd.DataFrame(grid.cv_results_).sort_values(by='rank_test_neg_log_loss')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_forest__bootstrap,param_forest__ccp_alpha,param_forest__class_weight,param_forest__criterion,param_forest__max_depth,param_forest__max_features,param_forest__max_leaf_nodes,param_forest__max_samples,param_forest__min_impurity_decrease,param_forest__min_samples_leaf,param_forest__min_samples_split,param_forest__min_weight_fraction_leaf,param_forest__n_estimators,param_forest__n_jobs,param_forest__oob_score,param_forest__random_state,param_forest__verbose,param_forest__warm_start,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_train_accuracy,split1_train_accuracy,split2_train_accuracy,split3_train_accuracy,split4_train_accuracy,mean_train_accuracy,std_train_accuracy,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_train_f1,split1_train_f1,split2_train_f1,split3_train_f1,split4_train_f1,mean_train_f1,std_train_f1,split0_test_neg_log_loss,split1_test_neg_log_loss,split2_test_neg_log_loss,split3_test_neg_log_loss,split4_test_neg_log_loss,mean_test_neg_log_loss,std_test_neg_log_loss,rank_test_neg_log_loss,split0_train_neg_log_loss,split1_train_neg_log_loss,split2_train_neg_log_loss,split3_train_neg_log_loss,split4_train_neg_log_loss,mean_train_neg_log_loss,std_train_neg_log_loss,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_train_precision,split1_train_precision,split2_train_precision,split3_train_precision,split4_train_precision,mean_train_precision,std_train_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_train_recall,split1_train_recall,split2_train_recall,split3_train_recall,split4_train_recall,mean_train_recall,std_train_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_train_roc_auc,split1_train_roc_auc,split2_train_roc_auc,split3_train_roc_auc,split4_train_roc_auc,mean_train_roc_auc,std_train_roc_auc
44,9.588275,0.757163,0.558042,0.100988,True,0.0,"{0: 1, 1: 1}",log_loss,None,0.472215,None,None,0.0,1,2,0.0,1000,-1,False,42,10,True,"{'forest__bootstrap': True, 'forest__ccp_alpha': 0.0, 'forest__class_weight': {0: 1, 1: 1}, 'forest__criterion': 'log_loss', 'forest__max_depth': None, 'forest__max_features': 0.4722149251619493, 'forest__max_leaf_nodes': None, 'forest__max_samples': None, 'forest__min_impurity_decrease': 0.0, 'forest__min_samples_leaf': 1, 'forest__min_samples_split': 2, 'forest__min_weight_fraction_leaf': 0.0, 'forest__n_estimators': 1000, 'forest__n_jobs': -1, 'forest__oob_score': False, 'forest__random_state': 42, 'forest__verbose': 10, 'forest__warm_start': True}",0.943548,0.862903,0.934959,0.910569,0.967480,0.923892,0.035513,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.820513,0.604651,0.809524,0.702703,0.904762,0.768430,0.104049,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-0.188384,-0.246548,-0.227353,-0.230374,-0.194891,-0.217510,0.022206,1,-0.063184,-0.057840,-0.060752,-0.061706,-0.062456,-0.061188,0.001858,0.941176,0.619048,0.850000,0.812500,0.904762,0.825497,0.112299,16,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.727273,0.590909,0.772727,0.619048,0.904762,0.722944,0.112945,7,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.983957,0.940508,0.943069,0.956349,0.953081,0.955393,0.015464,9,1.0,1.0,1.0,1.0,1.0,1.0,4.965068e-17
40,6.326918,0.050977,0.544000,0.015184,True,0.0,"{0: 1, 1: 1}",log_loss,None,0.623298,None,None,0.0,1,2,0.0,1000,-1,False,42,10,True,"{'forest__bootstrap': True, 'forest__ccp_alpha': 0.0, 'forest__class_weight': {0: 1, 1: 1}, 'forest__criterion': 'log_loss', 'forest__max_depth': None, 'forest__max_features': 0.6232981268275579, 'forest

In [19]:
grid.best_params_

{'forest__bootstrap': True,
 'forest__ccp_alpha': 0.0,
 'forest__class_weight': {0: 1, 1: 1},
 'forest__criterion': 'log_loss',
 'forest__max_depth': None,
 'forest__max_features': 0.4722149251619493,
 'forest__max_leaf_nodes': None,
 'forest__max_samples': None,
 'forest__min_impurity_decrease': 0.0,
 'forest__min_samples_leaf': 1,
 'forest__min_samples_split': 2,
 'forest__min_weight_fraction_leaf': 0.0,
 'forest__n_estimators': 1000,
 'forest__n_jobs': -1,
 'forest__oob_score': False,
 'forest__random_state': 42,
 'forest__verbose': 10,
 'forest__warm_start': True}